# Messages
Messages are the fundamental unit of context for models in LangChain. They represent the input and output of models, carrying both the content and metadata needed to represent the state of a conversation when interacting with an LLM.

## Basic Usage

In [2]:
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langchain.chat_models import init_chat_model

model = init_chat_model("openai:gpt-5-nano")


In [3]:
system_msg = SystemMessage("You are a full-stack comedian")

In [6]:
from langchain.agents import create_agent

agent = create_agent(
    model=model,
    tools=[],
    prompt=system_msg
)

In [7]:
human_msg = HumanMessage("Hello, how are you?")

result = agent.invoke({"messages": [human_msg]})

In [8]:
print(result["messages"][-1].content)

Hi there! I’m doing great—the coffee is strong, the code is compiling, and the jokes are buffering nicely. Front-end is polished, back-end is robust, and the database finally learned to commit.

What can I help you with today? Want a quick joke, some coding help, or a tiny stand-up bit about your project?


In [9]:
print(type(result["messages"][-1]))

<class 'langchain_core.messages.ai.AIMessage'>


In [10]:
for msg in result["messages"]:
    print(f"{msg.type}: {msg.content}\n")

human: Hello, how are you?

ai: Hi there! I’m doing great—the coffee is strong, the code is compiling, and the jokes are buffering nicely. Front-end is polished, back-end is robust, and the database finally learned to commit.

What can I help you with today? Want a quick joke, some coding help, or a tiny stand-up bit about your project?



> question: Why don't I see the system message here?

### Altenative formats
#### Strings

In [11]:
agent = create_agent(
    model=model,
    tools=[],
    prompt="You are a sports poet"  # SystemMessage
)

#### Dictionaries

In [12]:
result = agent.invoke(
    {"messages": {"role": "user", "content": "Write a haiku about sprinters"}}
)
print(result["messages"][-1].content)

Lunging at the line
Every breath a mirrored wind
Time loosens its hold


There are multiple roles:
```python
messages = [
    {"role": "system", "content": "You are a sports poetry expert who completes haikus that have been started"},
    {"role": "user", "content": "Write a haiku about sprinters"},
    {"role": "assistant", "content": "Feet don't fail me..."}
]
```

# OLD

In [ ]:
from langchain.chat_models import init_chat_model
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage

model = init_chat_model("openai:gpt-5-nano")

system_msg = SystemMessage("You are a helpful assistant.")
human_msg = HumanMessage("Hello, how are you?")

# Use with chat models
messages = [system_msg, human_msg]
response = model.invoke(messages)  # Returns AIMessage

In [ ]:
print(type(response))

In [ ]:
print(response.text)

## Formats

### Text Prompt
Text prompts are strings - ideal for straightforward generation tasks where you don’t need to retain conversation history.

In [ ]:
response = model.invoke("Write a haiku about spring")

In [ ]:
print(response.text)

### Message Prompts

In [ ]:
system_msg = SystemMessage("You are a helpful sports poet")
ai_msg = AIMessage("You're right, my rhymes are sublime.")
human_msg = HumanMessage("Write a haiku about baseball")
messages = [system_msg, ai_msg, human_msg]

In [ ]:
response = model.invoke(messages)

In [ ]:
print(response.text)

#### An alternate form of messages is a dictionary format.

In [ ]:
messages = [
    {"role": "system", "content": "You are a sports poetry expert who completes haikus that have been started"},
    {"role": "user", "content": "Write a haiku about sprinters"},
    {"role": "assistant", "content": "Feet don't fail me..."}
]

In [ ]:
response = model.invoke(messages)

In [ ]:
print(response.text)

## Printing
You will often see 'pretty printing' in this course.

In [ ]:
system_msg = SystemMessage("You are a helpful sports poet")
print (system_msg)   # is system_msg.__str__()

In [ ]:
print(system_msg.pretty_print())

In [ ]:
print(system_msg.model_dump())  # all fields

In [ ]:
print(type(system_msg))

## Content

In [ ]:
from langchain_core.messages import HumanMessage

# String content
human_message = HumanMessage("Hello, how are you?")

# Provider-native format (e.g., OpenAI)
human_message = HumanMessage(content=[
    {"type": "text", "text": "Hello, how are you?"},
    {"type": "image_url", "image_url": {"url": "https://example.com/image.jpg"}}
])

# List of standard content blocks
human_message = HumanMessage(content_blocks=[
    {"type": "text", "text": "Hello, how are you?"},
    {"type": "image", "url": "https://example.com/image.jpg"},
])

In [ ]:
from IPython.display import Image

# Just give it a file path
Image("./assets/Graphic.png",width=200)

In [ ]:
import base64
from langchain_core.messages import SystemMessage, HumanMessage

# Encode local image as base64
filename = "./assets/Graphic.png"
with open(filename, "rb") as f:
    image_bytes = f.read()
image_b64 = base64.b64encode(image_bytes).decode("utf-8")

# Messages
system_msg = SystemMessage("You are a helpful image reader!")
human_msg = HumanMessage(content=[
    {"type": "text", "text": "Please, tell me what this image means."},
    {"type": "image_url", "image_url": {"url": f"data:image/png;base64,{image_b64}"}}
])

messages = [system_msg, human_msg]

# Call GPT-5 (vision-capable)
response = model.invoke(messages, model="gpt-5")

print(response.content)

## AIMessage fields
AI messages from an LLM contain several useful fields beyond the content field.

In [ ]:
response = model.invoke("Write a haiku about spring")
response

In [ ]:
response.content

In [ ]:
response.response_metadata

In [ ]:
response.usage_metadata

## Content

In [ ]:
from langchain_core.messages import HumanMessage

# String content
human_message = HumanMessage("Hello, how are you?")

# Provider-native format (e.g., OpenAI)
human_message = HumanMessage(content=[
    {"type": "text", "text": "Hello, how are you?"},
    {"type": "image_url", "image_url": {"url": "https://example.com/image.jpg"}}
])

# List of standard content blocks
human_message = HumanMessage(content_blocks=[
    {"type": "text", "text": "Hello, how are you?"},
    {"type": "image", "url": "https://example.com/image.jpg"},
])

In [ ]:
from langchain.chat_models import init_chat_model
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage

model = init_chat_model("openai:gpt-5-nano")

In [ ]:
from IPython.display import Image

# Just give it a file path
Image("./assets/Graphic.png",width=200)

In [ ]:
import base64
from langchain_core.messages import SystemMessage, HumanMessage

# Encode local image as base64
filename = "./assets/Graphic.png"
with open(filename, "rb") as f:
    image_bytes = f.read()
image_b64 = base64.b64encode(image_bytes).decode("utf-8")

# Messages
system_msg = SystemMessage("You are a helpful image reader!")
human_msg = HumanMessage(content=[
    {"type": "text", "text": "Please, tell me what this image means."},
    {"type": "image_url", "image_url": {"url": f"data:image/png;base64,{image_b64}"}}
])

messages = [system_msg, human_msg]

# Call GPT-5 (vision-capable)
response = model.invoke(messages, model="gpt-5")

print(response.content)

In [ ]:
### Content Blocks

In [ ]:
# Messages
system_msg = SystemMessage("You are a helpful image reader!")
human_msg = HumanMessage(content_blocks=[
    {"type": "text", "text": "Please, tell me what this image means."},
    {"type": "image", "base64": image_b64, "mime_type": "image/png"}
])

messages = [system_msg, human_msg]

# Call GPT-5 (vision-capable)
response = model.invoke(messages, model="gpt-5")

print(response.content)

In [ ]:
from langchain.agents import create_agent
from langchain_core.messages import SystemMessage

#llm = init_chat_model("claude-3-5-sonnet-latest", model_provider="anthropic")
llm = init_chat_model("gpt-5", model_provider="openai")

agent = create_agent(
    model=llm,
    tools=[],
    prompt=SystemMessage(content="You are a funny guy"),
)

In [ ]:
for step in agent.stream(
    {"messages": [{"role": "user", "content": question}]},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

In [ ]:
result = agent.invoke(
    {"messages": [{"role": "user", 
                   "content": "Tell me a joke"}]}
    )
print(result["messages"][1])

In [ ]:
print(result["messages"][1])

In [ ]:
# Stream = values
for step in agent.stream(
    {"messages": [{"role": "user", 
                   "content": "Tell me a joke"}]},
    stream_mode="values",
    ):
    step["messages"][-1].pretty_print()

In [ ]:
for token, metadata in agent.stream(
    {"messages": [{"role": "user", "content": "What is the weather in SF?"}]},
    stream_mode="messages",
):
    print(f"node: {metadata['langgraph_node']}")
    print(f"content: {token.content_blocks}")
    print("\n")